# Scraping Music (... I guess)

In [1]:
import os
import base64
import requests
from datetime import datetime, timezone
from pymongo import MongoClient
from bs4 import BeautifulSoup
from configs import MUSIC_MAP, MUSIC_LINK, MONGO_URI

In [2]:
SAVE_DIR = "db"
os.makedirs(SAVE_DIR, exist_ok=True)

In [3]:
def get_composer_link(composer:str) -> str:
    return MUSIC_LINK + f'/{composer.lower()}.htm'

In [4]:
def get_genre_by_composer(composer, music_map):
    composer = composer.title()
    for genre, composers in music_map.items():
        if composer in composers:
            return genre
    return None  # If the composer is not found

In [5]:
def get_soup(url):
    print(f"🌐 Visiting {url}")
    response = requests.get(url)
    return BeautifulSoup(response.content, "html.parser")

In [6]:
def insert_into_mongo(documents, db_name="midi_db", collection_name="music_pieces"):
    client = MongoClient(MONGO_URI)
    db = client[db_name]
    collection = db[collection_name]

    # Optionally add timestamps to each document
    for doc in documents:
        doc["upload_date"] = datetime.now(timezone.utc).isoformat()

    result = collection.insert_many(documents)
    print(f"✅ Inserted {len(result.inserted_ids)} documents into {db_name}.{collection_name}")

In [7]:
def drop_collection(db_name="midi_db", collection_name="music_pieces"):
    client = MongoClient(MONGO_URI)
    db = client[db_name]

    if collection_name in db.list_collection_names():
        db.drop_collection(collection_name)
        print(f"🗑️ Dropped collection: {db_name}.{collection_name}")
    else:
        print(f"⚠️ Collection {db_name}.{collection_name} does not exist.")

In [8]:
def encode_mid_to_base64(filepath):
    """
    Reads a .mid file and returns a base64-encoded string.
    """
    with open(filepath, "rb") as f:
        encoded = base64.b64encode(f.read()).decode("utf-8")
    return encoded

In [9]:
def decode_base64_to_mid(encoded_str, output_path):
    """
    Decodes a base64 string and writes the binary data to a .mid file.
    """
    with open(output_path, "wb") as f:
        f.write(base64.b64decode(encoded_str.encode("utf-8")))

In [10]:
def files_are_equal(file1, file2):
    with open(file1, "rb") as f1, open(file2, "rb") as f2:
        return f1.read() == f2.read()

In [11]:
def drink_soup(soup, composer, save_dir="db"):
    """
    Downloads all .mid files found in the provided BeautifulSoup object
    and stores them as base64-encoded strings.

    Args:
        soup (BeautifulSoup): Parsed HTML content.
        composer (str): Composer's name for tagging metadata.
        save_dir (str): Directory where .mid files will be saved temporarily.

    Returns:
        list of dict: Metadata for each downloaded file, with base64 MIDI content.
    """
    period = get_genre_by_composer(composer, MUSIC_MAP)
    os.makedirs(save_dir, exist_ok=True)
    mid_links = soup.find_all("a", href=lambda href: href and href.endswith(".mid"))
    print(f"🎯 Found {len(mid_links)} MIDI files")

    downloaded_files = []

    for link in mid_links:
        href = link["href"]
        file_url = href if href.startswith("http") else f"https://www.midiworld.com{href}"
        filename = os.path.basename(file_url)
        save_path = os.path.join(save_dir, filename)

        try:
            print(f"⬇️ Downloading {filename}...")
            r = requests.get(file_url)
            with open(save_path, "wb") as f:
                f.write(r.content)
            print(f"✅ Saved to {save_path}")

            encoded = encode_mid_to_base64(save_path)

            downloaded_files.append({
                "filename": filename,
                "piece_name": link.get_text(strip=True),
                "composer": composer,
                "music_period": period,
                "source_url": file_url,
                "encoded_piece": encoded,
                "downloaded": True
            })

        except Exception as e:
            print(f"❌ Failed to download {file_url}: {e}")
            downloaded_files.append({
                "filename": filename,
                "piece_name": link.get_text(strip=True),
                "composer": composer,
                "music_period": period,
                "source_url": file_url,
                "encoded_piece": None,
                "downloaded": False,
                "error": str(e)
            })

    return downloaded_files

The ideal approach will be to iterate using the contents of the music map in the configs to align with the base url to scrape using beautiful soup.

In [12]:
composer = 'bach'
url1 = get_composer_link(composer)

In [13]:
genre = get_genre_by_composer(composer, MUSIC_MAP)
print(f"{composer} is from the {genre} period.")

bach is from the Baroque period.


In [ ]:
soup = get_soup(url1)

🌐 Visiting https://www.midiworld.com/bach.htm


In [15]:
downloaded_files = drink_soup(soup, composer)

🎯 Found 287 MIDI files
⬇️ Downloading bwv772.mid...
✅ Saved to db/bwv772.mid
⬇️ Downloading bwv773.mid...
✅ Saved to db/bwv773.mid
⬇️ Downloading bwv774.mid...
✅ Saved to db/bwv774.mid
⬇️ Downloading bwv775.mid...
✅ Saved to db/bwv775.mid
⬇️ Downloading bwv776.mid...
✅ Saved to db/bwv776.mid
⬇️ Downloading bwv777.mid...
✅ Saved to db/bwv777.mid
⬇️ Downloading bwv778.mid...
✅ Saved to db/bwv778.mid
⬇️ Downloading bwv779.mid...
✅ Saved to db/bwv779.mid
⬇️ Downloading bwv780.mid...
✅ Saved to db/bwv780.mid
⬇️ Downloading bwv781.mid...
✅ Saved to db/bwv781.mid
⬇️ Downloading bwv782.mid...
✅ Saved to db/bwv782.mid
⬇️ Downloading bwv783.mid...
✅ Saved to db/bwv783.mid
⬇️ Downloading bwv784.mid...
✅ Saved to db/bwv784.mid
⬇️ Downloading bwv785.mid...
✅ Saved to db/bwv785.mid
⬇️ Downloading bwv786.mid...
✅ Saved to db/bwv786.mid
⬇️ Downloading bwv787.mid...
✅ Saved to db/bwv787.mid
⬇️ Downloading bwv788.mid...
✅ Saved to db/bwv788.mid
⬇️ Downloading bwv789.mid...
✅ Saved to db/bwv789.mid
⬇️ Do

In [16]:
insert_into_mongo(downloaded_files)

✅ Inserted 287 documents into midi_db.music_pieces


In [21]:
file1="bwv772.mid"
test_audio_path = "db/"+file1
test_decoded_path = "db/decoded_"+file1
encoded = encode_mid_to_base64(test_audio_path)
decoded = decode_base64_to_mid(encoded, test_decoded_path)
# validate encoding/decoding approach
if files_are_equal("db/bwv772.mid", "db/decoded_bwv772.mid"):
    print(f"✅ Base and Decode match for {file1}")
else:
    print(f"❌ Base and Decode does not match for {file1}")

✅ Base and Decode match for bwv772.mid
